In [8]:
import os
import re
import spacy 
from spacy import displacy
import json
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd

from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models

In [9]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'\W*\b(?!no)\w{1,2}\b', '', text)
    stop_words = set(stopwords.words('english'))
    stop_words.update(('appellant', 'respondent', 'plaintiff', 'defendant', 'mr', 'dr', 'mdm', 'court','version', 'hr', 'would', 'case', 'sghc', 'court', 'sgca', 'also', 'first', 'person', 'statement', 'line', 'para', 'fact', 'one', 'may', 'time', 'could'))
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return words

In [10]:
# filepath = 'C:/Users/gaoh7/Documents/GitHub/case-outcome-predictor/data/rawish_data/facts.json'
filepath = '../data/rawish_data/facts.json'
print("Preprocessing text")
data = []
with open(filepath, 'r') as file:
    for line in file:
        data.append(json.loads(line))
df = pd.DataFrame(data)
df = df[['casename', 'facts']]    
df['processed_facts'] = df['facts'].apply(preprocess_text)
df.drop(columns=['facts'], inplace=True)
print(df["processed_facts"])
print("Finished preprocessing text")

Preprocessing text


0       [fact, widow, tan, geok, tee, deceased, sue, c...
1       [fact, surrounding, circumstance, including, a...
2       [background, appellant, french, company, secon...
3       [background, microsoft, adobe, autodesk, compa...
4       [fact, mere, assertion, suffice, exh, said, st...
                              ...                        
8515    [fact, accused, low, sze, song, low, year, old...
8516    [fact, giving, opinion, representation, amount...
8517    [fact, party, karan, bagga, litigant, proceedi...
8518                                                   []
8519    [fact, party, towa, company, incorporated, jap...
Name: processed_facts, Length: 8520, dtype: object
Finished preprocessing text


In [11]:
texts, article = [], []
print("Performing topic modelling")
for fact in df['processed_facts']:
    for word in fact:
        article.append(word)
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
# lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
# print(lda_model.show_topics())

best_coherence = -1
best_lda = None
for num_topics in range(5, 26, 5):
    # Train LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=num_topics,
                                                random_state=42,
                                                update_every=1,
                                                chunksize=100,
                                                passes=5,
                                                alpha='auto',
                                                per_word_topics=True)
    # Compute coherence score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()
    print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")
    if coherence_score > best_coherence:
        best_coherence = coherence_score
        best_topic = num_topics
print(f"Best no of topic: {best_topic} and Best Coherence Score: {best_coherence}")

Performing topic modelling
Number of topics: 5, Coherence Score: 0.4455967100032388
Number of topics: 15, Coherence Score: 0.5367434354666969
Number of topics: 25, Coherence Score: 0.5280405881368955
Best Coherence Score: 0.5367434354666969
Best no of topic: 15


In [12]:

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=dictionary,
                                            num_topics=best_topic,
                                            random_state=42,
                                            update_every=1,
                                            chunksize=100,
                                            passes=5,
                                            alpha='auto',
                                            per_word_topics=True)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.175851  0.056492       1        1  16.000024
9      0.107898  0.083775       2        1  15.578202
6     -0.089900  0.065606       3        1  11.865095
7      0.201607 -0.051632       4        1   8.079114
10    -0.072594  0.136292       5        1   7.754032
14    -0.119759  0.069182       6        1   6.905794
12    -0.095615  0.083667       7        1   6.641319
5      0.185046 -0.122308       8        1   4.677926
0      0.128260  0.065814       9        1   4.427843
11    -0.145845 -0.081072      10        1   4.383742
4      0.154559 -0.223852      11        1   3.350896
8     -0.010681  0.035790      12        1   3.317691
2     -0.134461  0.013543      13        1   3.276271
3     -0.043751  0.157922      14        1   2.362244
13    -0.240616 -0.289220      15        1   1.379807, topic_info=           Term          Freq         Total Category  logprob  loglift
1742   property  24947.000000  24947.000000  Default  30.0000  30.0000
815     accused  25918.000000  25918.000000  Default  29.0000  29.0000
589         ltd  31852.000000  31852.000000  Default  28.0000  28.0000
485   affidavit  22115.000000  22115.000000  Default  27.0000  27.0000
108     company  19398.000000  19398.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
1110      agent    577.519221   3695.484075  Topic15  -4.7765   2.4271
1804      class    522.567299   3433.391414  Topic15  -4.8765   2.4007
1057        apr    550.379582   9131.486266  Topic15  -4.8247   1.4744
2042      china    413.020701   1951.987568  Topic15  -5.1118   2.7301
384        sale    420.752239  10385.821717  Topic15  -5.0932   1.0771

[1036 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
5215      9  0.993920  abetment
5217      9  0.991641  abetting
5813      3  0.994600  academic
4272      8  0.614818  accident
4272     11  0.384780  accident
...     ...       ...       ...
2468      3  0.001364      yong
2468      6  0.023182      yong
2468      9  0.079547      yong
2468     11  0.005000      yong
2334      9  0.975699     youth

[3126 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 10, 7, 8, 11, 15, 13, 6, 1, 12, 5, 9, 3, 4, 14])

In [13]:
# filepath = 'C:/Users/gaoh7/Documents/GitHub/case-outcome-predictor/issues.json'
filepath = '../data/prediction_data/issues.json'
print("Preprocessing text")
data = []
with open(filepath, 'r') as file:
    for line in file:
        data.append(json.loads(line))
df = pd.DataFrame(data)
df = df[['casename', 'issues']]    
df['processed_issues'] = df['issues'].apply(preprocess_text)
df.drop(columns=['issues'], inplace=True)
print(df["processed_issues"])
print("Finished preprocessing text")

Preprocessing text
0       [claim, dismissed, cost, high, decision, fook,...
1       [claim, decision, appeal, brought, background,...
2       [appeal, question, arise, appeal, follows, app...
3       [appeal, assistant, registrar, ground, judgmen...
4       [appeal, january, dismissed, give, reason, evi...
                              ...                        
8515    [issue, sub, issue, arise, consideration, whet...
8516    [claim, conspiracy, defraud, fault, ken, sally...
8517    [issue, relating, defence, justification, qual...
8518                                                   []
8519    [background, dispute, towa, commenced, suit, a...
Name: processed_issues, Length: 8520, dtype: object
Finished preprocessing text


In [14]:
texts, article = [], []
print("Performing topic modelling")
for fact in df['processed_issues']:
    for word in fact:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
# lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
# print(lda_model.show_topics())

best_coherence = -1
best_lda = None

for num_topics in range(5, 26, 5):
    # Train LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=num_topics,
                                                random_state=42,
                                                update_every=1,
                                                chunksize=100,
                                                passes=5,
                                                alpha='auto',
                                                per_word_topics=True)
    
    # Compute coherence score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()
    
    print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")
    if coherence_score > best_coherence:
        best_coherence = coherence_score
        best_topic = num_topics
print(f"Best no of topic: {best_topic} and Best Coherence Score: {best_coherence}")

Performing topic modelling
Number of topics: 5, Coherence Score: 0.42097859259663306
Number of topics: 15, Coherence Score: 0.4989923642950815
Number of topics: 25, Coherence Score: 0.5152077879327654
Best Coherence Score: 0.5152077879327654
Best no of topic: 25


In [15]:

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=dictionary,
                                            num_topics=best_topic,
                                            random_state=42,
                                            update_every=1,
                                            chunksize=100,
                                            passes=5,
                                            alpha='auto',
                                            per_word_topics=True)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

KeyboardInterrupt: 